# TowerScout Demo Notebook

This notebook contains a basic script to stand up and run the pre-trained TowerScout model on a small sample of aerial images.  [TowerScout](https://groups.ischool.berkeley.edu/TowerScout/) was developed by a group a UC Berkeley to detect cooling towers from aerial imagery to assist public health officials with Legionella investigations and preparedness.  The original [TowerScout GitHub repository](https://github.com/TowerScout/TowerScout) can be referenced for more information about the project and their model training process.

## Getting Started
### Prerequisites

This demo uses python and a Jupyter notebook. It is recommended to use Python 3.7 or greater. Visit [python.org](https://www.python.org/downloads/) and [jupyter.org](https://jupyter.org/install) for installation instructions.

_PyTorch has great instructions on how to [get started](https://pytorch.org/get-started/locally/) with these fundamental prerequisites._

Create a python virtual environment with conda or venv.

conda:

  1. `conda create --name demo python=3.7`
  1. `activate demo`

venv:

  1. `python -m venv .env`
  1. `source ./env/bin/activate`

PyTorch must be installed before this notebook can be run. Visit [pytorch.org](https://pytorch.org/get-started/locally/) for instructions.

For initial testing, UGRC used a CPU-only install with conda on windows.

- `conda install pytorch torchvision cpuonly -c pytorch`

And a pip install on MacOS

1. `pip install torch torchvision`

YOLOv5 need to be installed for the TowerScout weights to be applied. Quick start instructions for YOLOv5 can be found on [GitHub](https://github.com/ultralytics/yolov5)

1. `git clone https://github.com/ultralytics/yolov5`
2. `cd yolov5`
3. `pip install -r requirements.txt`

The sample images should be tiled to an appropriate tile size for processing by YOLOv5 (UGRC used 512x512 pixel tiles).

In [1]:
# Import python libraries
from pathlib import Path
from time import perf_counter
import torch

### Load TowerScout

In [3]:
current_path = Path().resolve()

# Set up path to the yolov5 directory and local repo
yolov5_src_path = current_path / 'yolov5'

# Set up path to model weights file and load the TowerScout model
towerscout_weights_path = current_path / 'tower_scout' / 'xl_250_best.pt'

model = torch.hub.load(str(yolov5_src_path), 'custom', path=str(towerscout_weights_path), source='local')

YOLOv5  2022-6-8 Python-3.7.11 torch-1.10.2 CPU

Fusing layers... 
Model summary: 476 layers, 87198694 parameters, 0 gradients
Adding AutoShape... 


### Run TowerScout
If more than 100 images are in the tile directory, you may need to break the images into batches of 100 images to avoid a memory allocation error.  Results will be saved in the `\runs\detect\exp` directory.

In [4]:
# Set location of image tiles and cd into that directory
tile_folder = current_path / 'image_tiles'

# Build list of images in tile directory
images = list(tile_folder.glob('**/*.jpg'))

image_count = len(images)
start = perf_counter()

for i in range(0, image_count, 100):
    batch_start = perf_counter()
    batch = images[i:i+100]

    results = model(images)
    print(f'Time elapsed ({len(batch)} images): {perf_counter() - batch_start:.2f}s')

    results.print()
    results.save()

print(f'Total time elapsed ({image_count} images): {perf_counter() - start:.2f}s')

Time elapsed (19 images): 56.14s
image 1/19: 512x512 (no detections)
image 2/19: 512x512 1 ct
image 3/19: 512x512 3 cts
image 4/19: 512x512 2 cts
image 5/19: 512x512 1 ct
image 6/19: 512x512 4 cts
image 7/19: 512x512 5 cts
image 8/19: 512x512 1 ct
image 9/19: 512x512 1 ct
image 10/19: 512x512 1 ct
image 11/19: 512x512 1 ct
image 12/19: 512x512 1 ct
image 13/19: 512x512 3 cts
image 14/19: 512x512 1 ct
image 15/19: 512x512 1 ct
image 16/19: 512x512 2 cts
image 17/19: 512x512 1 ct
image 18/19: 512x512 2 cts
image 19/19: 512x512 1 ct
Speed: 12.6ms pre-process, 2940.7ms inference, 0.7ms NMS per image at shape (19, 3, 640, 640)


Saved 19 images to runs\detect\exp6


Total time elapsed (19 images): 56.36s
